# BoxRetriever

This will help you getting started with the Box [retriever](/docs/concepts/#retrievers). For detailed documentation of all BoxRetriever features and configurations head to the [API reference](https://python.langchain.com/v0.2/api_reference/box/retrievers/langchain_box.retrievers.box.BoxRetriever.html).

# Overview

The `BoxRetriever` class helps you get your unstructured content from Box in Langchain's `Document` format. You can do this by searching for files based on a full-text search or using Box AI to retrieve a `Document` containing the result of an AI query against files. This requires including a `List[str]` containing Box file ids, i.e. `["12345","67890"]` 

:::info
Box AI requires an Enterprise Plus license
:::

Files without a text representation will be skipped.

### Integration details

1: Bring-your-own data (i.e., index and search a custom corpus of documents):

| Retriever | Self-host | Cloud offering | Package |
| :--- | :--- | :---: | :---: |
[BoxRetriever](https://python.langchain.com/v0.2/api_reference/box/retrievers/langchain_box.retrievers.box.BoxRetriever.html) | ❌ | ✅ | langchain-box |

## Setup

In order to use the Box package, you will need a few things:

* A Box account — If you are not a current Box customer or want to test outside of your production Box instance, you can use a [free developer account](https://account.box.com/signup/n/developer#ty9l3).
* [A Box app](https://developer.box.com/guides/getting-started/first-application/) — This is configured in the [developer console](https://account.box.com/developers/console), and for Box AI, must have the `Manage AI` scope enabled. Here you will also select your authentication method
* The app must be [enabled by the administrator](https://developer.box.com/guides/authorization/custom-app-approval/#manual-approval). For free developer accounts, this is whomever signed up for the account.

### Credentials

For these examples, we will use [token authentication](https://developer.box.com/guides/authentication/tokens/developer-tokens). This can be used with any [authentication method](https://developer.box.com/guides/authentication/). Just get the token with whatever methodology. If you want to learn more about how to use other authentication types with `langchain-box`, visit the [Box provider](/docs/integrations/providers/box) document.

In [1]:
import getpass
import os

box_developer_token = getpass.getpass("Enter your Box Developer Token: ")

Enter your Box Developer Token:  ········


If you want to get automated tracing from individual queries, you can also set your [LangSmith](https://docs.smith.langchain.com/) API key by uncommenting below:

In [ ]:
# os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter your LangSmith API key: ")
# os.environ["LANGSMITH_TRACING"] = "true"

### Installation

This retriever lives in the `langchain-box` package:

In [ ]:
%pip install -qU langchain-box

## Instantiation

Now we can instantiate our retriever:

## Search

In [2]:
from langchain_box import BoxRetriever

retriever = BoxRetriever(box_developer_token=box_developer_token)

## Box AI

In [ ]:
from langchain_box import BoxRetriever

box_file_ids = ["1514555423624", "1514553902288"]

retriever = BoxRetriever(
    box_developer_token=box_developer_token, box_file_ids=box_file_ids
)

## Usage

In [3]:
query = "victor"

retriever.invoke(query)

[Document(metadata={'source': 'https://dl.boxcloud.com/api/2.0/internal_files/1233039227512/versions/1346280085912/representations/extracted_text/content/', 'title': 'FIVE_FEET_AND_RISING_by_Peter_Sollett_pdf'}, page_content='\n3/20/23, 5:31 PM FIVE FEET AND RISING by Peter Sollett\n\nwww.dailyscript.com/scripts/fivefeetandrising.html 1/25\n\nFIVE FEET AND RISING\n\nby\n\nPeter Sollett\n\nFADE IN:\n\nEXT. 8TH STREET BETWEEN AVENUES C AND D - DAY \n\nA group of dark-skinned girls wearing cheerleading outfits \nalign themselves in formation on the sidewalk. They begin to \ndance. No music can be heard. The sound of the girls\' bodies \nis our soundtrack. We hear their strained breathing, palms \nand sneaker bottoms pounding while they hum and count softly \nto themselves in an effort to keep the rhythm.\n\nSLO-MO: We explore the bodies of the dancers; their bright \neyes and sweaty brows, their stomping feet and colliding \nhands (dark side and light side). The younger girls perform \npr

## Use within a chain

Like other retrievers, BoxRetriever can be incorporated into LLM applications via [chains](/docs/how_to/sequence/).

We will need a LLM or chat model:

```{=mdx}
import ChatModelTabs from "@theme/ChatModelTabs";

<ChatModelTabs customVarName="llm" />
```

In [5]:
# | output: false
# | echo: false

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [6]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

retriever = BoxRetriever(box_developer_token=box_developer_token, character_limit=10000)

context = (
    "You are an actor reading scripts to learn about your role in an upcoming movie."
)
question = "describe the character Victor"

prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context provided.

    Context: {context}

    Question: {question}"""
)


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [7]:
chain.invoke("victor")

'Victor is a skinny 12-year-old with sloppy hair who is seen sleeping on his fire escape in the sun. He is hesitant to go to the pool with his friend Carlos because he is afraid of getting in trouble for not letting his mother cut his hair. Ultimately, he decides to go to the pool with Carlos.'

## API reference

For detailed documentation of all BoxRetriever features and configurations head to the [API reference](https://python.langchain.com/v0.2/api_reference/box/retrievers/langchain_box.retrievers.box.BoxRetriever.html).


## Help

If you have questions, you can check out our [developer documentation](https://developer.box.com) or reach out to use in our [developer community](https://community.box.com).